In [22]:
import re
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import nltk

from tqdm import tqdm
from string import punctuation
from sklearn.metrics import f1_score

from dostoevsky.tokenization import RegexTokenizer
from flashtext import KeywordProcessor

Попытка ускорить спейси с 30 минут с помощью конфигурирования пайплайна:

In [2]:
sns.set_theme()
tqdm.pandas()

nlp = spacy.load(
    'ru_core_news_sm',
    enable=['ner', 'tokenizer'],
    exclude=['tok2vec', 'attribute_ruler', 'morphologizer', 'senter', 'lemmatizer']
)
nlp.config

c:\Users\sveta\AppData\Local\Programs\Python\Python39\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'ru_core_news_sm' (3.5.0) was trained with spaCy v3.5.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


{'paths': {'train': None, 'dev': None, 'vectors': None, 'init_tok2vec': None},
 'system': {'gpu_allocator': None, 'seed': 0},
 'nlp': {'lang': 'ru',
  'pipeline': ['parser', 'ner'],
  'disabled': ['parser'],
  'before_creation': None,
  'after_creation': None,
  'after_pipeline_creation': None,
  'batch_size': 256,
  'tokenizer': {'@tokenizers': 'spacy.Tokenizer.v1'}},
 'components': {'parser': {'factory': 'parser',
   'learn_tokens': False,
   'min_action_freq': 30,
   'model': {'@architectures': 'spacy.TransitionBasedParser.v2',
    'state_type': 'parser',
    'extra_state_tokens': False,
    'hidden_width': 64,
    'maxout_pieces': 2,
    'use_upper': True,
    'nO': None,
    'tok2vec': {'@architectures': 'spacy.Tok2VecListener.v1',
     'width': '${components.tok2vec.model.encode:width}',
     'upstream': 'tok2vec'}},
   'moves': None,
   'scorer': {'@scorers': 'spacy.parser_scorer.v1'},
   'update_with_oracle_cut_size': 100},
  'ner': {'factory': 'ner',
   'incorrect_spans_key': 

In [3]:
DATA_PATH = 'data'
RANDOM_STATE = 42
COMPANY_REGEX = r'".+"'

In [4]:
df = pd.read_pickle(DATA_PATH + '/mentions texts.pickle')
df.head()

,ChannelID,messageid,issuerid,MessageID,DateAdded,DatePosted,MessageText,IsForward
0,1197210433,5408,90,5408,2021-02-06 01:42:42,2020-04-29 07:29:01,?? Фокус недели #ФН Сегодня ????? ММК опублик...,False
1,1203560567,64803,57,64803,2021-02-06 01:47:00,2020-01-21 12:51:42,??#LSRG ЛСР - операционные результаты (2019г)...,False
2,1197210433,23389,152,23389,2021-07-21 13:46:31,2021-07-21 11:15:46,#CHMF Северсталь (CHMF) впервые поставила в Бр...,False
3,1066174394,677,112,677,2021-09-21 04:23:59,2016-12-16 10:00:04,"""Версия: Многоходовочка по Роснефти Роснефтег...",False
4,1239405989,4486,115,4486,2023-01-20 15:02:22,2023-01-20 13:03:38,🟢 Новости к этому часу ⚪️ФРС США необходимо ...,False


Семплируем тестовые данные с запасом

In [5]:
samples = df.sample(1100, random_state=RANDOM_STATE)
samples.head()

,ChannelID,messageid,issuerid,MessageID,DateAdded,DatePosted,MessageText,IsForward
3725,1203560567,52298,187,52298,2021-02-06 01:47:00,2019-09-24 16:47:30,????????#крипто #франция #внедрение К началу 2...,False
13914,1565800335,1955,150,1955,2023-11-12 07:46:15,2023-11-12 07:30:00,​🏦 Сбербанк (SBER) - чистая продолжает бить ре...,False
9347,1203560567,55570,175,55570,2021-02-06 01:47:00,2019-10-22 11:31:05,??#TRNFP #бонды ТРАНСНЕФТЬ ПЕРЕНЕСЛА СБОР ЗАЯВ...,False
7527,1203560567,227127,7,227127,2023-01-30 15:01:09,2023-01-30 11:32:10,"""🇷🇺#RTKM #VTBR #цифровизация #россия """"Ростел...",False
4336,1351339368,736,7,736,2021-02-06 19:06:57,2020-03-30 09:56:53,Доброе утро! ?? В пятницу Федрезерв США объя...,False


После оптимизации пайплайна спейси обрабатывает 1100 текстов за ~25-30 секунд

In [6]:
samples['spacy'] = samples['MessageText'].progress_apply(nlp)

100%|██████████| 1100/1100 [00:24<00:00, 45.25it/s]


In [7]:
samples['sentences'] = samples['MessageText'].progress_apply(nltk.sent_tokenize)

100%|██████████| 1100/1100 [00:00<00:00, 1573.47it/s]


Попробуем оптимизировать нер, воспользовавшись префиксным деревом. Для этого подготовим данные:

In [8]:
synonyms_mapper = pd.read_excel(DATA_PATH + f'/names and synonyms.xlsx', index_col='issuerid')
synonyms_mapper

,EMITENT_FULL_NAME,VeryOddCompany,BGTicker,BGTicker.1,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
issuerid,,,,,,,,,,,,,,
1,"Акционерный коммерческий банк ""Держава"" публич...",NaN,NaN,DERZP,Держава,DERZHAVA,DERZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"""МОСКОВСКИЙ КРЕДИТНЫЙ БАНК"" (публичное акционе...",NaN,CBOM RX,NaN,Московский кредитный банк,мкб,Credit Bank of Moscow,Credit Bank,NaN,NaN,NaN,NaN,NaN,NaN
3,"""Российский акционерный коммерческий дорожный ...",NaN,NaN,RDRB,Российский акционерный коммерческий дорожный банк,РДБанк,Дорожный банк,Russian public joint-stock commercial roads Bank,RosDorBank,РосДорБанк,roads Bank,NaN,NaN,NaN
4,"Акционерная компания ""АЛРОСА"" (публичное акцио...",NaN,ALRS RX,NaN,алроса,alrosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Акционерный Коммерческий банк ""АВАНГАРД"" - пуб...",NaN,NaN,AVAN,Авангард,AVANGARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,Henderson,NaN,NaN,HNFG,Хендерсон,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
271,Совкомбанк,NaN,NaN,SVCB,Sovcombank,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
272,ЕвроТранс,NaN,NaN,EUTR,АЗС Трасса,"АЗС ""Трасса""",АЗС «Трасса»,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
synonyms_mapper['extended'] = synonyms_mapper['BGTicker'].apply(lambda x: x.split()[0] if not pd.isna(x) else x)

synonyms_mapper['extended_2'] = synonyms_mapper['EMITENT_FULL_NAME'].apply(
    lambda x: re.findall(COMPANY_REGEX, x)[0] if re.findall(COMPANY_REGEX, x) else None
)

In [10]:
synonyms_cols = ['EMITENT_FULL_NAME', 'VeryOddCompany', 'BGTicker', 'BGTicker.1',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'extended', 'extended_2']

synonyms_list = synonyms_mapper[synonyms_cols].values
synonyms_list

array([['Акционерный коммерческий банк "Держава" публичное акционерное общество',
        nan, nan, ..., nan, nan, '"Держава"'],
       ['"МОСКОВСКИЙ КРЕДИТНЫЙ БАНК" (публичное акционерное общество)',
        nan, 'CBOM RX', ..., nan, 'CBOM', '"МОСКОВСКИЙ КРЕДИТНЫЙ БАНК"'],
       ['"Российский акционерный коммерческий дорожный банк" (публичное акционерное общество)',
        nan, nan, ..., nan, nan,
        '"Российский акционерный коммерческий дорожный банк"'],
       ...,
       ['ЕвроТранс', nan, nan, ..., nan, nan, None],
       ['Делимобиль', nan, nan, ..., nan, nan, None],
       ['Диасофт', nan, nan, ..., nan, nan, None]], dtype=object)

In [24]:
synonyms_list = [[f'{synonym.strip()}' for synonym in synonyms_list if not pd.isna(synonym)] for synonyms_list in synonyms_list]
keyword_dict = dict(zip(synonyms_mapper.index.values, synonyms_list))

keyword_dict

{1: ['Акционерный коммерческий банк "Держава" публичное акционерное общество',
  'DERZP',
  'Держава',
  'DERZHAVA',
  'DERZ',
  '"Держава"'],
 2: ['"МОСКОВСКИЙ КРЕДИТНЫЙ БАНК" (публичное акционерное общество)',
  'CBOM RX',
  'Московский кредитный банк',
  'мкб',
  'Credit Bank of Moscow',
  'Credit Bank',
  'CBOM',
  '"МОСКОВСКИЙ КРЕДИТНЫЙ БАНК"'],
 3: ['"Российский акционерный коммерческий дорожный банк" (публичное акционерное общество)',
  'RDRB',
  'Российский акционерный коммерческий дорожный банк',
  'РДБанк',
  'Дорожный банк',
  'Russian public joint-stock commercial roads Bank',
  'RosDorBank',
  'РосДорБанк',
  'roads Bank',
  '"Российский акционерный коммерческий дорожный банк"'],
 4: ['Акционерная компания "АЛРОСА" (публичное акционерное общество)',
  'ALRS RX',
  'алроса',
  'alrosa',
  'ALRS',
  '"АЛРОСА"'],
 5: ['Акционерный Коммерческий банк "АВАНГАРД" - публичное акционерное общество',
  'AVAN',
  'Авангард',
  'AVANGARD',
  '"АВАНГАРД"'],
 6: ['акционерный коммерческ

Обучим префиксное дерево распознавать организации:

In [12]:
keyword_preprocessor = KeywordProcessor()
keyword_preprocessor.add_keywords_from_dict(keyword_dict)

In [ ]:
samples['keywords'] = samples['MessageText'].progress_apply(keyword_preprocessor.extract_keywords).apply(lambda x: list(set(x)))

In [14]:
samples[['issuerid', 'keywords']]

,issuerid,keywords
3725,187,[]
13914,150,[150]
9347,175,[175]
7527,7,"[142, 7]"
4336,7,[7]
...,...,...
6733,220,"[100, 230, 7, 231, 235, 175, 220]"
1297,225,"[225, 227, 99, 230, 236, 141, 142, 47, 112, 11..."
9422,25,"[89, 163, 111, 112, 48, 115, 150, 25, 27, 127]"
18342,44,[44]


In [15]:
samples['predict'] = samples.apply(lambda x: x['issuerid'] if x['issuerid'] in x['keywords'] else 4242, axis=1)

In [16]:
samples[samples['predict'] == 4242].shape

(63, 12)

Среднее время выполнения:

In [19]:
%%timeit

samples['keywords'] = samples['MessageText'].apply(keyword_preprocessor.extract_keywords).apply(lambda x: list(set(x)))

1.2 s ± 125 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


F1 score считаем следующим образом: если нашли упоминание компании issuerid, то 1, иначе 0

In [17]:
f1_score(samples['issuerid'], samples['predict'], average='macro')

0.9219246355250649

Проверим на всем датасете:

In [20]:
df['keywords'] = df['MessageText'].progress_apply(keyword_preprocessor.extract_keywords).apply(lambda x: list(set(x)))
df['predict'] = df.apply(lambda x: x['issuerid'] if x['issuerid'] in x['keywords'] else 4242, axis=1)

f1_score(df['issuerid'], df['predict'], average='macro')

100%|██████████| 19355/19355 [00:30<00:00, 644.63it/s] 


0.9441922260844381

In [21]:
30 / 19335 * 1000

1.5515903801396431